### Exercise 7 - Association Rules

First name: Brian
<br>
Last name: Schweigler
<br>
Matriculation number: 16-102-071

#### (1) Problem: Decide whether to wait for a table at a restaurant, based on various attributes.


We will use the same classifiers as in last series, Decision Tree, KNN, Naive Bayes

In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from mlxtend.evaluate import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn import svm
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("data/restaurant.csv")
pd.set_option('display.max_colwidth', None)
le = preprocessing.LabelEncoder()
x_train = df[["choice", "bar", "day", "hungry", "patron", "price", "rain", "booking", "type"]]
x_train = pd.DataFrame(columns=x_train.columns, data=le.fit_transform(x_train.values.flatten()).reshape(x_train.shape))
x_train["time"] = df["time"]

y = le.fit(df["wait"])
y = le.transform(df["wait"])

Xd_train, Xd_test, y_train, y_test = train_test_split(x_train, y, test_size=0.4)

print(Xd_train)
print("y_train = ", y_train, "\n")

print(Xd_test)
print("y_test = ", y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
    choice  bar  day  hungry  patron  price  rain  booking  type  time
2        4    4    3       3       9      0     3        3    10     0
10       3    3    3       3       8      0     3        3    11     0
9        4    4    4       4       6      2     3        4     7    20
5        3    4    3       4       9      1     4        3     7     0
3        4    3    4       4       6      0     3        3    11    20
8        3    4    4       3       6      0     4        3    10    60
11       4    4    4       4       6      0     3        3    10    40
y_train =  [1 0 0 1 1 0 1] 

   choice  bar  day  hungry  patron  price  rain  booking  type  time
7       3    3    3       4       9      1     4        4    11     0
6       3    4    3       3       8      0     4        3    10    20
4       4    3    4       3       6      2     3        4     5    60
0       4    3    3       4       9

In [26]:
# Create a svm classifier using one kernel (linear, polynomial, and radial basis)
clf = svm.SVC(kernel='linear')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with linear kernel = ", y_test)
print("y_pred with linear kernel = ", y_pred, "\n")
print("NB_Accuracy with linear kernel = ", NB_Accuracy, "\n")
print("Confusion Matrix with linear kernel \n", confusion_matrix(y_test, y_pred))

y_test with linear kernel =  [1 0 0 0 1]
y_pred with linear kernel =  [0 1 1 1 1] 

NB_Accuracy with linear kernel =  0.2 

Confusion Matrix with linear kernel 
 [[0 3]
 [1 1]]


In [27]:
clf = svm.SVC(kernel='rbf')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with rbf kernel = ", y_test)
print("y_pred with rbf kernel  = ", y_pred, "\n")
print("NB_Accuracy with rbf kernel  = ", NB_Accuracy, "\n")
print("Confusion Matrix with rbf kernel \n", confusion_matrix(y_test, y_pred))

y_test with rbf kernel =  [1 0 0 0 1]
y_pred with rbf kernel  =  [1 1 1 1 1] 

NB_Accuracy with rbf kernel  =  0.4 

Confusion Matrix with rbf kernel 
 [[0 3]
 [0 2]]


In [28]:
clf = svm.SVC(kernel='poly')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with poly kernel = ", y_test)
print("y_pred with poly kernel = ", y_pred, "\n")
print("NB_Accuracy with poly kernel = ", NB_Accuracy, "\n")
print("Confusion Matrix with poly kernel \n", confusion_matrix(y_test, y_pred))

y_test with poly kernel =  [1 0 0 0 1]
y_pred with poly kernel =  [1 1 1 1 1] 

NB_Accuracy with poly kernel =  0.4 

Confusion Matrix with poly kernel 
 [[0 3]
 [0 2]]


#### Co-occurence matrix

In [ ]:
cooc_mat = x_train.T.dot(x_train)
cooc_mat

#### Calculate support, confidence, completeness, lift, and leverage for the following rules. Build the co-occurrence matrix. (again?)

#### Explain these measures

Support: The fraction of the total number of data entries in which the variable occurs.

Confidence: The conditional probability of occurrence of consequent given the antecedent.

Completeness: The ratio the overall transactions where the predicted item appear that is covered by the rule.

Lift: Controls for the support (frequency) of consequent while calculating the conditional probability of occurrence of Y given X.

Leverage: Leverage measures the difference of XX and YY appearing together in the data set and what would be expected if XX and YY were statistically dependent.

#### Use the Apriori algorithm to find frequent item sets. We are only interested in item sets having a support value of at least 50%.

In [58]:
# Inspiration: https://www.kaggle.com/code/sangwookchn/association-rule-learning-with-scikit-learn
from apyori import apriori
rules = apriori(x_train[['choice', 'bar', 'day', 'hungry', 'patron', 'rain', 'booking', 'type']], min_support = 0.5, min_confidence = 0.1)
results = list(rules)
results = pd.DataFrame(results)
results.head(5)

,items,support,ordered_statistics
0,(a),0.5,"[((), (a), 0.5, 1.0)]"
1,(n),0.5,"[((), (n), 0.5, 1.0)]"
2,(r),0.5,"[((), (r), 0.5, 1.0)]"
